In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


house_data = pd.read_csv('kc_house_data.csv')

independent_v = house_data.drop(['price','date', 'id'], axis =1)
dependent_v = house_data['price']


In [2]:
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual
import ipywidgets as widgets


# At first lets see all the feature we have in our dataset

@interact
def show_info(col = independent_v.head()):
    return independent_v.loc[independent_v[col]]


def f(price):
    return price

interact(f, price= dependent_v.head(100));

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


interactive(children=(Dropdown(description='col', options=('bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',…

interactive(children=(Dropdown(description='price', options=(221900.0, 538000.0, 180000.0, 604000.0, 510000.0,…

In [3]:
## UI design ##
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


def linear_r(test_size):
    independent_v = house_data.drop(['price','date', 'id'], axis =1)
    #independent_v = house_data.drop(['price','date', 'id', 'bedrooms', 'bathrooms', 'condition', 'grade','lat','long'], axis =1)
    #independent_v = house_data.drop(['price','date', 'id', 'condition', 'grade','lat','long', 'waterfront','view','yr_renovated', 'zipcode'], axis =1)
    #print(independent_v.head(10))

    dependent_v = house_data['price']
    #print(dependent_v.head(10))

    independent_train, independent_test, dependent_train, dependent_test = train_test_split(independent_v, dependent_v, test_size = test_size,random_state = 20)


    l_regression = LinearRegression()
    l_regression.fit(independent_train, dependent_train)

    print("interecpt is", l_regression.intercept_)
    #print("coeffiecient for each features is", l_regression.coef_)
    print("Score for training is: ",l_regression.score(independent_train,dependent_train))
    
    #print("score for training is: ", score1)
    print("Score for testing is: ",l_regression.score(independent_test,dependent_test))


# Now we are gonna predict y using  x feature test and train data 
    dependent_predict = l_regression.predict(independent_test)
    error_val = mean_squared_error(dependent_test, dependent_predict)
    e1 = np.sqrt(error_val)
    print("error is", e1)


    plt.figure(figsize=(8, 4))
    plt.scatter(dependent_test, dependent_predict, color = 'red')
    plt.plot(dependent_predict, dependent_predict, ls="-", c=".10")
    plt.title('Actual-values VS Predicted-values')
    plt.xlabel('actualVal')
    plt.ylabel('predicteVal')
    plt.show()
    
    #return(print("score for training is",score1), score2)

In [4]:
interact(linear_r, test_size = widgets.Dropdown(options = {"10% data": 0.1, "25% data": 0.25, "30% data": 0.3, "40% data": 0.4, "50% data": 0.5, "60% data": 0.6},description="Test set size"));


interactive(children=(Dropdown(description='Test set size', options={'10% data': 0.1, '25% data': 0.25, '30% d…

In [5]:
# UI desdgin For logistic regression ##

def price(price_amount):
    price_high = np.array(house_data['price'])

    list1 = []
    
    for price in price_high:
        if price >= price_amount:     ##estimating the price by setting the threshold
            list1.append(1) # yes, price is high
        else:
            list1.append(0) # NO, price is not high


    house_data['price_high'] = list1 

    total = house_data['price_high'].value_counts()
    print("Total 0's ans 1's we have: \n", total)

    sns.countplot(x = 'price_high', data = house_data, palette = 'hls')
    plt.show()
    
    
    pd.crosstab(house_data.yr_built, house_data.price_high).plot(kind='bar',figsize=(18,5) )
    plt.title('yr_built frequency')
   


In [6]:
interact(price, price_amount = widgets.Dropdown(options =[250000,500000, 600000, 650000, 950000, 1500000], description = 'select price:' ));


interactive(children=(Dropdown(description='select price:', options=(250000, 500000, 600000, 650000, 950000, 1…

In [7]:
### Data preprocess for logistic regression model ###

all_x = house_data.drop([ 'price_high', 'date'], axis =1)
y_val = house_data['price_high']

l_dataframe = pd.DataFrame(all_x)
x_independent_all = l_dataframe.drop(['waterfront', 'condition', 'lat', 'long', 'yr_renovated'], axis = 1)  

In [8]:
#### UI design for logistic####
### before improving the model ###

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def logis(test_size):
    
    
    feature_train, feature_test, target_train, target_test = train_test_split(x_independent_all, y_val, test_size = test_size,random_state = 20)

    logistic = LogisticRegression()
    logistic.fit(feature_train, target_train)

    target_predict = logistic.predict(feature_test)

    print('Accuracy of logistic regression classifier on test set:', logistic.score(feature_test, target_test))
   
    confu_matrix = confusion_matrix(target_test, target_predict)


    axis = sns.heatmap(pd.DataFrame(confu_matrix), annot=confu_matrix, cmap="Blues" ,fmt='g')
    bottom, top = axis.get_ylim()
    axis.set_ylim(bottom + 0.5, top - 0.5)
    plt.tight_layout()
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

In [9]:
interact(logis, test_size = widgets.RadioButtons(options = {"10% data": 0.1, "25% data": 0.25, "30% data": 0.3, "40% data": 0.4, "50% data": 0.5, "60% data": 0.6},description="Test set size"));


interactive(children=(RadioButtons(description='Test set size', options={'10% data': 0.1, '25% data': 0.25, '3…

In [10]:
### IMPROVING MODEL BY DOING MIN_MAX_SCALER ###
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(x_independent_all)


In [11]:
### UI design after improving the model ###
def logis_improved(test_size):
    features_train, features_test, targets_train, targets_test = train_test_split(X_scale, y_val, test_size = test_size,random_state = 20)
    #print(feature_train)

    logistic = LogisticRegression()
    logistic.fit(features_train, targets_train)

    targets_predict = logistic.predict(features_test)

    print('Accuracy of logistic regression classifier on test set:', logistic.score(features_test, targets_test))
    print(classification_report(targets_test, targets_predict))

    confus_matrix = confusion_matrix(targets_test, targets_predict)


    axis = sns.heatmap(pd.DataFrame(confus_matrix), annot=confus_matrix, cmap="Blues" ,fmt='g')
    bottom, top = axis.get_ylim()
    axis.set_ylim(bottom + 0.5, top - 0.5)
    plt.tight_layout()
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')


In [12]:
interact(logis_improved, test_size = widgets.RadioButtons(options = {"10% data": 0.1, "25% data": 0.25, "30% data": 0.3, "40% data": 0.4, "50% data": 0.5, "60% data": 0.6},description="Test set size"));


interactive(children=(RadioButtons(description='Test set size', options={'10% data': 0.1, '25% data': 0.25, '3…

In [13]:
### UI design for Neural network ###
### after improved the model ###
import keras
from keras.models import Sequential
from keras.layers import Dense
def neural_improved(test_size):
    x_t, x_te, y_t, y_te = train_test_split(X_scale, y_val, test_size = test_size,random_state = 42)
    x_v, x_tes, y_v, y_tes = train_test_split(x_te, y_te, test_size = 0.5)

    model = Sequential()
    model.add(Dense(35, input_dim = 15, activation = 'relu'))
    model.add(Dense(35, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

          

    model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

    chart = model.fit(x_t, y_t, batch_size=20, epochs=10, validation_data=(x_v, y_v))
    
    print(model.evaluate(x_tes, y_tes))
    
    pred1 = model.predict(x_tes)
    pred1 = (pred1 > 0.5)

    confusion_m1 = confusion_matrix(y_tes, pred1)
    print(confusion_m1)

    print(classification_report(y_tes, pred1))
    
    plt.plot(chart.history['accuracy'])
    plt.plot(chart.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
    plt.plot(chart.history['loss'])
    plt.plot(chart.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    

Using TensorFlow backend.


In [14]:
interact(neural_improved, test_size = widgets.RadioButtons(options = {"10% data": 0.1, "25% data": 0.25, "30% data": 0.3, "40% data": 0.4, "50% data": 0.5, "60% data": 0.6},description="Test set size"));


interactive(children=(RadioButtons(description='Test set size', options={'10% data': 0.1, '25% data': 0.25, '3…